# Prepare training database

In [1]:
import Bio.SeqIO as SeqIO


LTR_sequences = [record for record in SeqIO.parse("/opt/xhorvat9_TE_DBs/CNN_BERT_train_DB/LTR_sequences.fasta", "fasta")]
#max_sequence_length = 250
non_LTR_sequences = []

### Create a sampling distribution from sequence lengths

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
seq_lens = [len(rec.seq) for rec in LTR_sequences if len(rec.seq) <= 2000]
sns.set(rc={'figure.figsize':(14,8)}, font_scale=1.4)

sns.histplot(seq_lens)
plt.title("Sequence Length Distribution",pad=24)
plt.ylabel("Count")
plt.xlabel("Sequence Length")

### 2. Generate random sequences
- let 25 % of the negative sequences be generated sequences

In [28]:
import random
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
import random 
sequence_id = 0 
n_seqs = int(len(LTR_sequences) * 0.25)
avg_len = int(sum([len(seq) for seq in LTR_sequences])/len(LTR_sequences))

for _ in range(n_seqs):
    length = random.choice(seq_lens)
    sequence = ''.join([random.choice(["A","C", "T", "G"]) for _ in range(length)])
    record = SeqRecord(
        Seq(sequence),
        id=f"{sequence_id}_non-LTR_generated",
        description="randomly generated non-LTR sequence")
    sequence_id += 1

    non_LTR_sequences.append(record)

### 3. take sequences cut out from genomic sequences 

In [1]:
# Identify regions without TE content
genomes_path = "/opt/xhorvat9_TE_DBs/Genomes/Genomes/"
annotation_path = "/opt/xhorvat9_TE_DBs/Genomes/Genomes/all_annotations/"
genomes = ["Casuarina_equisetifolia",  "Citrullus_lanatus",  "Hordeum_vulgare",  "Juglans_regia",	"Zea_mays"]
genome_paths = [f"{genomes_path}{genome}.fa" for genome in genomes]
annotation_paths = [f"{annotation_path}{genome}.txt" for genome in genomes]

#### Identify LTR locations

In [ ]:
import tqdm
import pandas as pd
LTRs = {}
for genome in genomes:
    annot = pd.read_csv(f"{annotation_path}{genome}.txt", sep="\t", low_memory=False).iloc[:-1, :-1]
    #LTR_IDs = set(annot[annot["Domain"].str.contains("ltr")]["LTR_ID"])
    LTR_IDs = set(annot["LTR_ID"])
    
    for idx in tqdm.tqdm(LTR_IDs):
        element = annot[annot["LTR_ID"] == idx]
        element_5LTR = element[element["Domain"] == "intact_5ltr"][["Start", "End"]]
        element_3LTR = element[element["Domain"] == "intact_3ltr"][["Start", "End"]]
        try:
            LTRs[idx] = ({"Start": int(element_5LTR["Start"]),"End": int(element_5LTR["End"])},{"Start": int(element_3LTR["Start"]),"End": int(element_3LTR["End"])})
        except:
            pass

In [5]:
import Bio.SeqIO as SeqIO
chromosome_sequences = {(rec.id): rec.seq for rec in SeqIO.parse("/opt/xhorvat9_TE_DBs/Genomes/Genomes/Zea_mays.fa", "fasta")}

In [31]:
def check_interval(LTRs, chromosome, start, end):
    ltr_starts = LTRs[LTRs["Chromosome"].str.match(chromosome)].Start
    ltr_ends = LTRs[LTRs["Chromosome"].str.match(chromosome)].End
    ii = pd.IntervalIndex.from_arrays(ltr_starts, ltr_ends, closed="both")
    return ii.contains(start) | ii.contains(end)

In [32]:
sequence_dicts = dict([(genome, dict([(record.id, record.seq) for record in SeqIO.parse(f"{genomes_path}{genome}.fa", "fasta")])) for genome in genomes])

In [33]:
LTRs = {}
for genome in genomes:
    annot = pd.read_csv(f"{annotation_path}{genome}.txt", sep="\t", low_memory=False)
    LTRs[genome] = annot[annot["Domain"].str.contains("ltr")]

#### Extract the sequences avoiding the LTR locations and regions with N count above 10%

In [34]:
import random
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
n_sequences = int(len(LTR_sequences) * 0.60)

for i in range(n_sequences):
    # Randomly sample genome
    chosen_genome = random.choice(genomes)
    # randomly sample chromosome
    chosen_chromosome = random.choice(list(sequence_dicts[chosen_genome].keys()))
    # randomly sample length
    chosen_length = random.choice(seq_lens)
    if len(sequence_dicts[chosen_genome][chosen_chromosome])-chosen_length < 50:
        i -= 1
        continue
    random_start = random.randint(0, len(sequence_dicts[chosen_genome][chosen_chromosome])-chosen_length)
    while any(check_interval(LTRs[chosen_genome], chosen_chromosome, random_start, random_start+chosen_length)):
        random_start = random.randint(0, len(sequence_dicts[chosen_genome][chosen_chromosome])-chosen_length)
    seq = sequence_dicts[chosen_genome][chosen_chromosome][random_start:random_start+chosen_length]
    if seq.count("N") > len(seq)/10:
        i -= 1
        continue
    record = SeqRecord(
        Seq(seq),
        id=f"{sequence_id}_non-LTR_genome_extract",
        description=f"{chosen_genome} chr{chosen_chromosome} {random_start}:{random_start+chosen_length}")
    sequence_id += 1
    non_LTR_sequences.append(record)

In [ ]:
SeqIO.write(non_LTR_sequences, "non_LTR_sequences.fasta", "fasta")